<a href="https://colab.research.google.com/github/claralanda/TFG_Clara_Landaribar/blob/main/Evaluacion_BLEU_DD_ED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Importación de las librerías:




In [ ]:
import nltk
import nltk.translate.bleu_score as bleu
import numpy as np
import pandas as pd
import os
import re
import json
import math
import re
import warnings; warnings.simplefilter('ignore')
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
from nltk import ngrams
from collections import Counter

try:
  nltk.data.find('tokenizers/punkt')
except LookupError:
  nltk.download('punkt')

try:
  nltk.data.find('tokenizers/punkt')
except LookupError:
  nltk.download('punkt')


Entrar en la carpeta de los documentos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My \Drive/Datos/BLEU

/content/drive/My Drive/Datos/BLEU


Extraer las bases de datos de Daily Dialog (DD)

In [ ]:
DD_reference_dataset = pd.read_csv("DAILYD_translation_en2es.csv")
DD_translation_dataset = pd.read_csv("DAILYD_translation_es2en.csv")

DD_reference = DD_reference_dataset['SEG']
DD_translation = DD_reference_dataset['translation']
DD_candidate = DD_translation_dataset['translation']
n_DD=len(DD_reference)
print(len(DD_reference))

102968


Extraer las bases de datos de Empathetic Dialogues (ED)

In [ ]:
ED_reference_dataset = pd.read_csv("MPATHY_translation_en2es.csv")
ED_translation_dataset = pd.read_csv("MPATHY_translation_es2en.csv")

ED_reference = ED_reference_dataset['SEG']
ED_translation = ED_reference_dataset['translation']
ED_candidate = ED_translation_dataset['translation']
n_ED=len(ED_reference)
print(len(ED_reference))

107220


Muestra por pantalla de una pareja ejemplo antes del preprocesado

In [ ]:
print('Esta es una frase de referencia de Daily Dialog sin preprocesado:' + DD_reference[20])
print('Esta es una frase candidata de Daily Dialog sin preprocesado:' + DD_candidate[20])
print('Esta es una frase de referencia de Empathetic Dialogues sin preprocesado: ' + ED_reference[20])
print('Esta es una frase candidata de Empathetic Dialogues sin preprocesado: ' + ED_candidate[20])

Esta es una frase de referencia de Daily Dialog sin preprocesado: What ’ s wrong ? Didn ’ t you think it was fun ? ! 
Esta es una frase candidata de Daily Dialog sin preprocesado: What ' s wrong ? Didn ' t you think it was funny ? ! 
Esta es una frase de referencia de Empathetic Dialogues sin preprocesado: What do you mean it hasn ' t been easy ? How close have you come to cheating ? 
Esta es una frase candidata de Empathetic Dialogues sin preprocesado: What do you mean , it hasn ' t been easy ? How close have you been to cheating ? 


Tokenización 

In [ ]:
DD_ref_list = [DD_reference[i].split() for i in range(n_DD)]
DD_can_list = [DD_candidate[i].split() for i in range(n_DD)]

ED_ref_list = [ED_reference[i].split() for i in range(n_ED)]
ED_can_list = [ED_candidate[i].split() for i in range(n_ED)]

Muestra por pantalla de una pareja de ejemplo tras tokenización de los datos antes del preprocesado

In [ ]:
print('Esta es una frase de referencia de Daily Dialog tras primer tokenizado:' + str(DD_ref_list[20]))
print('Esta es una frase candidata de Daily Dialog tras primer tokenizado:' + str(DD_can_list[20]))
print('Esta es una frase de referencia de Empathetic Dialogues tras primer tokenizado: ' + str(ED_ref_list[20]))
print('Esta es una frase candidata de Empathetic Dialogues tras primer tokenizado: ' + str(ED_can_list[20]))

Esta es una frase de referencia de Daily Dialog tras primer tokenizado:['What', '’', 's', 'wrong', '?', 'Didn', '’', 't', 'you', 'think', 'it', 'was', 'fun', '?', '!']
Esta es una frase candidata de Daily Dialog tras primer tokenizado:['What', "'", 's', 'wrong', '?', 'Didn', "'", 't', 'you', 'think', 'it', 'was', 'funny', '?', '!']
Esta es una frase de referencia de Empathetic Dialogues tras primer tokenizado: ['What', 'do', 'you', 'mean', 'it', 'hasn', "'", 't', 'been', 'easy', '?', 'How', 'close', 'have', 'you', 'come', 'to', 'cheating', '?']
Esta es una frase candidata de Empathetic Dialogues tras primer tokenizado: ['What', 'do', 'you', 'mean', ',', 'it', 'hasn', "'", 't', 'been', 'easy', '?', 'How', 'close', 'have', 'you', 'been', 'to', 'cheating', '?']


Adición de espacios delante y detrás de los signos de puntuación

In [ ]:
DD_candidate = [ re.sub('(?<! )(?=[.,!?()\'])|(?<=[.,!?()\'])(?! )', r' ', DD_candidate[i]) for i in range(n_DD)]
DD_reference = [ re.sub('(?<! )(?=[.,!?()\'])|(?<=[.,!?()\'])(?! )', r' ', DD_reference[i]) for i in range(n_DD)]

ED_candidate = [ re.sub('(?<! )(?=[.,!?()\'])|(?<=[.,!?()\'])(?! )', r' ', ED_candidate[i]) for i in range(n_ED)]
ED_reference = [ re.sub('(?<! )(?=[.,!?()\'])|(?<=[.,!?()\'])(?! )', r' ', ED_reference[i]) for i in range(n_ED)]

Muestra por pantalla de una pareja de ejemplo tras adición espacios

In [ ]:
print('Esta es una frase de referencia de Daily Dialog tras adición espacios:' + str(DD_reference[20]))
print('Esta es una frase candidata de Daily Dialog tras adición espacios:' + str(DD_candidate[20]))
print('Esta es una frase de referencia de Empathetic Dialogues tras adición espacios: ' + str(ED_reference[20]))
print('Esta es una frase candidata de Empathetic Dialogues tras adición espacios: ' + str(ED_candidate[20]))

Esta es una frase de referencia de Daily Dialog tras adición espacios: What ’ s wrong ? Didn ’ t you think it was fun ? ! 
Esta es una frase candidata de Daily Dialog tras adición espacios: What ' s wrong ? Didn ' t you think it was funny ? ! 
Esta es una frase de referencia de Empathetic Dialogues tras adición espacios: What do you mean it hasn ' t been easy ? How close have you come to cheating ? 
Esta es una frase candidata de Empathetic Dialogues tras adición espacios: What do you mean , it hasn ' t been easy ? How close have you been to cheating ? 


Tokenización tras adición espacios

In [ ]:
DD_ref_list_2 = [DD_reference[i].split() for i in range(n_DD)]
DD_can_list_2 = [DD_candidate[i].split() for i in range(n_DD)]

ED_ref_list_2 = [ED_reference[i].split() for i in range(n_ED)]
ED_can_list_2 = [ED_candidate[i].split() for i in range(n_ED)]

Muestra por pantalla de una pareja de ejemplos tras tokenización y adición de espacios

In [ ]:
print('Esta es una frase de referencia de Daily Dialog tras cambio apóstrofos:' + str(DD_ref_list_2[20]))
print('Esta es una frase candidata de Daily Dialog tras cambio apóstrofos:' + str(DD_can_list_2[20]))
print('Esta es una frase de referencia de Empathetic Dialogues tras cambio apóstrofos: ' + str(ED_ref_list_2[20]))
print('Esta es una frase candidata de Empathetic Dialogues tras cambio apóstrofos: ' + str(ED_can_list_2[20]))

Esta es una frase de referencia de Daily Dialog tras cambio apóstrofos:['What', '’', 's', 'wrong', '?', 'Didn', '’', 't', 'you', 'think', 'it', 'was', 'fun', '?', '!']
Esta es una frase candidata de Daily Dialog tras cambio apóstrofos:['What', "'", 's', 'wrong', '?', 'Didn', "'", 't', 'you', 'think', 'it', 'was', 'funny', '?', '!']
Esta es una frase de referencia de Empathetic Dialogues tras cambio apóstrofos: ['What', 'do', 'you', 'mean', 'it', 'hasn', "'", 't', 'been', 'easy', '?', 'How', 'close', 'have', 'you', 'come', 'to', 'cheating', '?']
Esta es una frase candidata de Empathetic Dialogues tras cambio apóstrofos: ['What', 'do', 'you', 'mean', ',', 'it', 'hasn', "'", 't', 'been', 'easy', '?', 'How', 'close', 'have', 'you', 'been', 'to', 'cheating', '?']


Cambio de los apóstrofos curvos por apóstrofos simples

In [ ]:
DD_candidate_2 = [re.sub(r'(\’)', '\'', DD_candidate[i]) for i in range(n_DD)]
DD_reference_2 = [re.sub(r'(\’)', '\'', DD_reference[i]) for i in range(n_DD)]

ED_candidate_2 = [re.sub(r'(\’)', '\'', ED_candidate[i]) for i in range(n_ED)]
ED_reference_2 = [re.sub(r'(\’)', '\'', ED_reference[i]) for i in range(n_ED)]

Muestra por pantalla de una pareja de ejemplos tras cambio apóstrofo

In [ ]:
print('Esta es una frase de referencia de Daily Dialog tras cambio apóstrofos:' + str(DD_reference_2[20]))
print('Esta es una frase candidata de Daily Dialog tras cambio apóstrofos:' + str(DD_candidate_2[20]))
print('Esta es una frase de referencia de Empathetic Dialogues tras cambio apóstrofos: ' + str(ED_reference_2[20]))
print('Esta es una frase candidata de Empathetic Dialogues tras cambio apóstrofos: ' + str(ED_candidate_2[20]))

Esta es una frase de referencia de Daily Dialog tras cambio apóstrofos: What ' s wrong ? Didn ' t you think it was fun ? ! 
Esta es una frase candidata de Daily Dialog tras cambio apóstrofos: What ' s wrong ? Didn ' t you think it was funny ? ! 
Esta es una frase de referencia de Empathetic Dialogues tras cambio apóstrofos: What do you mean it hasn ' t been easy ? How close have you come to cheating ? 
Esta es una frase candidata de Empathetic Dialogues tras cambio apóstrofos: What do you mean , it hasn ' t been easy ? How close have you been to cheating ? 


Tokenización tras cambio apóstrofo

In [ ]:
DD_ref_list_3 = [DD_reference_2[i].split() for i in range(n_DD)]
DD_can_list_3 = [DD_candidate_2[i].split() for i in range(n_DD)]

ED_ref_list_3 = [ED_reference_2[i].split() for i in range(n_ED)]
ED_can_list_3 = [ED_candidate_2[i].split() for i in range(n_ED)]

Muestra por pantalla de una pareja de ejemplo tras tokenización y cambio de apóstrofo

In [ ]:
print('Esta es una frase de referencia de Daily Dialog tras cambio apóstrofos:' + str(DD_ref_list_3[20]))
print('Esta es una frase candidata de Daily Dialog tras cambio apóstrofos:' + str(DD_can_list_3[20]))
print('Esta es una frase de referencia de Empathetic Dialogues tras cambio apóstrofos: ' + str(ED_ref_list_3[20]))
print('Esta es una frase candidata de Empathetic Dialogues tras cambio apóstrofos: ' + str(ED_can_list_3[20]))

Esta es una frase de referencia de Daily Dialog tras cambio apóstrofos:['What', "'", 's', 'wrong', '?', 'Didn', "'", 't', 'you', 'think', 'it', 'was', 'fun', '?', '!']
Esta es una frase candidata de Daily Dialog tras cambio apóstrofos:['What', "'", 's', 'wrong', '?', 'Didn', "'", 't', 'you', 'think', 'it', 'was', 'funny', '?', '!']
Esta es una frase de referencia de Empathetic Dialogues tras cambio apóstrofos: ['What', 'do', 'you', 'mean', 'it', 'hasn', "'", 't', 'been', 'easy', '?', 'How', 'close', 'have', 'you', 'come', 'to', 'cheating', '?']
Esta es una frase candidata de Empathetic Dialogues tras cambio apóstrofos: ['What', 'do', 'you', 'mean', ',', 'it', 'hasn', "'", 't', 'been', 'easy', '?', 'How', 'close', 'have', 'you', 'been', 'to', 'cheating', '?']


Resultados de BLUE de corpus DD

In [ ]:
refs, hyps = [], []

a= DD_ref_list_3
b= DD_can_list_3
for label, hyp in zip(a, b):
    ref_tokens = label
    hyp_tokens = hyp
    refs.append([ref_tokens])
    hyps.append(hyp_tokens)
bleu = corpus_bleu(refs, hyps,smoothing_function=SmoothingFunction().method1)
print (bleu)
  
report = '\n===== BLEU = %f =====\n' % (bleu,)

0.6949330377574379


Resultados de BLUE de corpus ED

In [ ]:
refs, hyps = [], []

a= ED_ref_list_3
b= ED_can_list_3
for label, hyp in zip(a, b):
    ref_tokens = label
    hyp_tokens = hyp
    refs.append([ref_tokens])
    hyps.append(hyp_tokens)
bleu = corpus_bleu(refs, hyps,smoothing_function=SmoothingFunction().method1)
print (bleu)
  
report = '\n===== BLEU = %f =====\n' % (bleu,)

0.5931373426595172


Ejemplos de BLEU en oraciones

In [ ]:
print(sentence_bleu(ED_ref_list_3[102958],ED_can_list_3[102958]))
print(sentence_bleu(DD_ref_list_3[13773], DD_can_list_3[13773]))
print(sentence_bleu(ED_ref_list[102958],ED_can_list[102958]))
print(sentence_bleu(DD_ref_list[13773], DD_can_list[13773]))
print(DD_can_list_3[13773])
print(DD_ref_list_3[13773])
print(ED_can_list_3[102958])
print(ED_ref_list_3[102958])
print(DD_can_list[13773])
print(DD_ref_list[13773])
print(ED_can_list[102958])
print(ED_ref_list[102958])

In [ ]:
cuenta=0
score_total=0
for x in range(len(DD_ref_list_3)):
  score=sentence_bleu([DD_ref_list_3[x]],DD_can_list_3[x])
  if score == 1 and score == 1 :
    print(x,DD_reference[x],DD_translation[x],DD_candidate[x],score)
    score_total=score_total+len(DD_reference[x])
    cuenta=cuenta+1
print(cuenta)
print(score_total/cuenta)

TypeError: ignored

In [ ]:
cuenta=0
score_total=0

for x in range(len(ED_ref_list_3)):
  score=sentence_bleu([ED_ref_list_3[x]],ED_can_list_3[x])
  if score >= 0.9 and score <= 1 :
    print(x,ED_reference[x],ED_translation[x],ED_candidate[x],score)
    score_total=score_total+len(ED_reference[x])
    cuenta=cuenta+1

print(cuenta)
print(score_total/cuenta)

Se han truncado las últimas 5000 líneas del flujo de salida.
57057 Guy friends for the most part Guy amigos en su mayor parte Guy friends for the most part 1.0
57074 Ha ha , that ' s a good one .   Ja ja, esa es buena.   Ha ha , that ' s a good one .   1.0
57089 I ' m so sorry to hear that . How long has it been if I may ask ?  Siento mucho oír eso. ¿Cuánto tiempo ha pasado si puedo preguntar? I ' m so sorry to hear that . How long has it been if I may ask ?  1.0
57096 I just found the most honest car mechanic .  Acabo de encontrar al mecánico de autos más honesto. I just found the most honest car mechanic .  1.0
57119 I graudated with a chemistry degree this year .  Este año me puse con un título en química. This year I got a degree in chemistry .  0.9036020036098448
57137 Oh , I ' m so excited for you ! I hope you get a raise too !  ¡Oh, estoy tan emocionada por ti! ¡Espero que también consigas un aumento! Oh , I ' m so excited for you ! I hope you get a raise too !  1.0
57155 I went

Pruebas para encontrar errores

In [ ]:
cuenta=0
score_total=0
for x in range(len(DD_ref_list_3)):
  score=sentence_bleu([DD_ref_list_3[x]],DD_can_list_3[x])
  score_total=score_total+score

print (score_total/len(DD_ref_list_3))

cuenta=0
score_total=0
for x in range(len(DD_ref_list_2)):
  score=sentence_bleu([DD_ref_list_2[x]],DD_can_list_2[x])
  score_total=score_total+score

print (score_total/len(DD_ref_list_2))

cuenta=0
score_total=0
for x in range(len(ED_ref_list_3)):
  score=sentence_bleu([ED_ref_list_3[x]],ED_can_list_3[x])
  score_total=score_total+score

print (score_total/len(ED_ref_list_3))

cuenta=0
score_total=0
for x in range(len(ED_ref_list)):
  score=sentence_bleu([ED_ref_list[x]],ED_can_list[x])
  score_total=score_total+score

print (score_total/len(ED_ref_list))

Ejemplos con oraciones aleatorias:

In [ ]:
ejemplo_ref=["Good", "idea"]
ejemplo_can=["Good", "idea"]
a=['do','you','want','to','use','chopsticks']
b=['do','you','like','to','use','chopsticks']
aa=['what','is','your','name','sir']
bb=['what','is','your','name','sir']
print(sentence_bleu([aa],bb,weights=(1,0,0,0)))
print(sentence_bleu([ejemplo_ref],ejemplo_can,weights=(0.25,0.25,0.25,0.25)))

1.0
1.0


Obtención de valoraciones BLEU de DD:

In [ ]:
res_vector=[]
for x in range(len(DD_reference)):
  bleu_score=sentence_bleu([DD_ref_list_3[x]],DD_can_list_3[x])
  print(x, bleu_score, DD_ref_list_3[x],DD_can_list_3[x]) 
  res_vector.append(bleu_score)

Creación de documento EXCEL con valoraciones BLEU de DD:

In [ ]:
df=pd.DataFrame(res_vector)
df.columns = ['puntuacion']
df.to_excel(excel_writer = "Bleu_scores_DD.xlsx")

Obtención de valoraciones BLEU de ED:

In [ ]:
res_vector=[]
for x in range(len(DD_reference)):
  bleu_score=sentence_bleu([ED_ref_list_3[x]],ED_can_list_3[x])
  print(x, bleu_score, ED_ref_list_3[x],ED_can_list_3[x]) 
  res_vector.append(bleu_score)

Streaming output truncated to the last 5000 lines.
97968 0.488923022434901 ['I', 'feel', 'sorry', 'for', 'me', 'too', '!'] ['I', 'feel', 'sorry', 'for', 'myself', 'too', '!']
97969 0.3203558799120807 ['Yeah', '.', 'Do', 'you', 'have', 'back', 'up', 'plans', 'for', 'this', 'issue', '?'] ['Yes', '.', 'Do', 'you', 'have', 'backup', 'plans', 'for', 'this', 'problem', '?']
97970 0.35930411196308415 ['They', 'are', 'very', 'helpful', 'and', 'I', 'always', 'will', 'be', 'faithful', 'to', 'them'] ['They', 'are', 'very', 'useful', 'and', 'I', 'will', 'always', 'be', 'faithful', 'to', 'them']
97971 1.0 ['That', "'", 's', 'nice', ',', 'I', "'", 'm', 'glad', 'you', 'feel', 'that', 'way', '.'] ['That', "'", 's', 'nice', ',', 'I', "'", 'm', 'glad', 'you', 'feel', 'that', 'way', '.']
97972 0.6652049901111008 ['Yes', ',', 'what', 'about', 'you', '?', 'Do', 'you', 'feel', 'likewise', '?'] ['Yes', ',', 'what', 'about', 'you', '?', 'Do', 'you', 'feel', 'the', 'same', 'way', '?']
97973 0.871953102945297 [

Creación de documento EXCEL con valoraciones BLEU de ED:

In [ ]:
df=pd.DataFrame(res_vector)
df.columns = ['puntuacion']
df.to_excel(excel_writer = "Bleu_scores_ED.xlsx")